In [5]:
import pandas as pd
import sqlalchemy as sa
import re

# --- CONFIGURATION ---
SERVER_NAME = r'Kelsey' # Use your specific server name
DATABASE_NAME = 'AIDA_Alberta_Econ'
EXCEL_PATH = r'C:\Users\kelse\OneDrive\Documents\School\Winter 2026\AIDA 2157\economic-review-indicators-data-tables.xlsx'

def run_warehouse_pipeline():
    try:
        # Connect to SQL Server
        connection_url = f"mssql+pyodbc://{SERVER_NAME}/{DATABASE_NAME}?driver=ODBC+Driver+17+for+SQL+Server&trusted_connection=yes"
        engine = sa.create_engine(connection_url)

        print("🚀 Loading Excel Workbook...")
        excel_file = pd.ExcelFile(EXCEL_PATH)
        all_sheets = excel_file.sheet_names

        for sheet in all_sheets:
            if "Cover" in sheet or "Contents" in sheet:
                continue

            print(f"--- 🛠️ Processing: {sheet} ---")
            
            # Read sheet (Header=4 skips the government titles)
            df = pd.read_excel(EXCEL_PATH, sheet_name=sheet, header=5)

            # CLEANING LOGIC: 
            # 1. Clean Table Name (Removes spaces/numbers for SQL compatibility)
            clean_table_name = re.sub(r'[^a-zA-Z]', '_', sheet).strip('_')
            clean_table_name = re.sub(r'_+', '_', clean_table_name)

            # 2. Clean Column Names (Removes symbols like $ or % that crash SQL)
            df.columns = [re.sub(r'[^a-zA-Z0-9]', '_', str(c)).strip('_') for c in df.columns]

            # Add this after your cleaning logic in Script 1
            # This keeps only columns that are NOT just numbers/dates
            df = df.loc[:, ~df.columns.str.contains('^Unnamed|^[0-9]', case=False)]

            # 3. Load to SQL
            if not df.empty:
                df.to_sql(clean_table_name, engine, if_exists='replace', index=False)
                print(f"✅ Created Table: [dbo].[{clean_table_name}]")

        print("\n🏆 WAREHOUSE COMPLETE!")
    except Exception as e:
        print(f"\n❌ FAILED: {str(e)}")

if __name__ == "__main__":
    run_warehouse_pipeline()

🚀 Loading Excel Workbook...
--- 🛠️ Processing: Table ---


c:\Users\kelse\miniforge3\envs\ml-env\Lib\site-packages\pandas\io\sql.py:1649: SAWarning: Unrecognized server version info '17.0.1000.7'.  Some SQL Server features may not function properly.
  con = self.exit_stack.enter_context(con.connect())


✅ Created Table: [dbo].[Table]
--- 🛠️ Processing: Time Series ---
--- 🛠️ Processing: 1) Oil Prices ---
--- 🛠️ Processing: 2) Economic Activity ---
--- 🛠️ Processing: 3) Labour 1 ---
--- 🛠️ Processing: 4) Labour 2  ---
--- 🛠️ Processing: 5) Business Output ---
--- 🛠️ Processing: 6) Rigs ---
--- 🛠️ Processing: 7) Housing ---
--- 🛠️ Processing: 8) Inflation, AWE & Retail ---
--- 🛠️ Processing: 9) Real GDP ---
✅ Created Table: [dbo].[Real_GDP]
--- 🛠️ Processing: 10) CPI ---
✅ Created Table: [dbo].[CPI]
--- 🛠️ Processing: 11) UnemploymentRate ---
✅ Created Table: [dbo].[UnemploymentRate]
--- 🛠️ Processing: 12) Employment ---
✅ Created Table: [dbo].[Employment]
--- 🛠️ Processing: 13) Avg Weekly Earnings ---
✅ Created Table: [dbo].[Avg_Weekly_Earnings]
--- 🛠️ Processing: 14) Housing ---
✅ Created Table: [dbo].[Housing]

🏆 WAREHOUSE COMPLETE!


1. The Pipeline: In the Python script, we used re.sub(r'[^a-zA-Z]', '_', sheet). Why is it critical to remove special characters from table names before sending them to SQL?
You need to remove the special characters becuase SQL uses specific characters for syntax and can get confused if we leave them in.
2. Automation: If the Alberta Government releases a new Excel file next month with 5 new tabs, what changes do you need to make to your Python script?
You should not have to make any changes to the python scriptnot have to make any manual changes to the core logic.
3. Connectivity: Explain the difference between SQLAlchemy (The Translator) and the ODBC Driver (The Tunnel).

4. Business Impact: How does having this data in a SQL Warehouse make it easier for the RDP-Pizza shop to predict their staffing needs compared to looking at a messy Excel file?
Moving from a "messy Excel file" to a SQL Warehouse transforms how a business like RDP-Pizza functions

In [6]:
import pandas as pd
import sqlalchemy as sa
import numpy as np
from sklearn.preprocessing import StandardScaler
from sklearn.neighbors import KNeighborsClassifier

# ==========================================
# 1. INITIALIZE & TRAIN
# ==========================================
SERVER_NAME = r'Kelsey'
DATABASE_NAME = 'AIDA_Alberta_Econ'

try:
    conn_str = f"mssql+pyodbc://{SERVER_NAME}/{DATABASE_NAME}?driver=ODBC+Driver+17+for+SQL+Server&trusted_connection=yes"
    engine = sa.create_engine(conn_str)

    # Pulling Gold View from SQL Warehouse
    df = pd.read_sql("SELECT * FROM vw_Pizza_Model_Data", engine).dropna()
    
    # Capture historical boundaries for the Guidance Note
    limits = {
        'inf_min': df['CPI_Inflation'].min(), 'inf_max': df['CPI_Inflation'].max(),
        'earn_min': df['Avg_Earnings'].min(), 'earn_max': df['Avg_Earnings'].max(),
        'grow_min': df['Retail_Growth'].min(), 'grow_max': df['Retail_Growth'].max()
    }

    # Prepare ML Model
    X = df[['CPI_Inflation', 'Avg_Earnings', 'Retail_Growth']]
    y = df['Econ_Status']

    scaler = StandardScaler()
    X_scaled = scaler.fit_transform(X)
    model = KNeighborsClassifier(n_neighbors=5).fit(X_scaled, y)

    print("✅ Supervised Trained ML Model 1: Online and Calibrated.")

# ==========================================
# 2. MODEL INTERACTION LOOP
# ==========================================
    while True:
        print("\n" + "="*65)
        print("🤖 SUPERVISED TRAINED ML MODEL 1: PREDICTOR")
        print("="*65)
        
        # --- THE UPDATED GUIDANCE NOTE ---
        print(f"📌 NOTE: Use inputs based on our Warehouse data points:")
        print(f"   👉 Inflation: {limits['inf_min']:.2f}% to {limits['inf_max']:.2f}%")
        print(f"   👉 Earnings:  ${limits['earn_min']:.2f} to ${limits['earn_max']:.2f}")
        print(f"   👉 Growth:    {limits['grow_min']:.2f}% to {limits['grow_max']:.2f}%")
        print("-" * 65)
        
        try:
            val_in = input("\nEnter Expected Inflation % (or 'exit'): ")
            if val_in.lower() == 'exit': break
            
            inf = float(val_in)
            earn = float(input("Enter Expected Avg Earnings: "))
            grow = float(input("Enter Expected Retail Growth %: "))

            # --- 🛡️ THE HARD-CAPPED GUARDRAILS ---
            if inf > 25 or inf < -10 or grow > 50 or grow < -50 or earn > 10000 or earn <= 0:
                print("\n❌ DATA ERROR: Input values are economically impossible.")
                print("The Supervised Model cannot predict on 'Sci-Fi' or Zero data.")
                continue

            # PREDICTION PHASE
            user_input = pd.DataFrame([[inf, earn, grow]], 
                                     columns=['CPI_Inflation', 'Avg_Earnings', 'Retail_Growth'])
            user_scaled = scaler.transform(user_input)
            prediction = model.predict(user_scaled)[0]

            print(f"\n✅ MODEL PREDICTION: {prediction}")

            # SAVE TO SQL
            save = input("Save this prediction to the tracking table? (y/n): ")
            if save.lower() == 'y':
                result_df = pd.DataFrame({
                    'Timestamp': [pd.Timestamp.now()],
                    'In_Inflation': [inf], 
                    'In_Earnings': [earn], 
                    'In_Growth': [grow],
                    'Model_Prediction': [prediction],
                    'Model_ID': ['Supervised_ML_Model_1']
                })
                result_df.to_sql('Model_Predictions', engine, if_exists='append', index=False)
                print("💾 Saved to [Model_Predictions] table.")

        except ValueError:
            print("❌ ERROR: Please enter numeric values only.")

except Exception as e:
    print(f"❌ CRITICAL SYSTEM ERROR: {e}")

print("\n👋 Model 1 Session Closed.")

❌ CRITICAL SYSTEM ERROR: Execution failed on sql 'SELECT * FROM vw_Pizza_Model_Data': (pyodbc.ProgrammingError) ('42S22', "[42S22] [Microsoft][ODBC Driver 17 for SQL Server][SQL Server]Invalid column name '2023_12_31_00_00_00'. (207) (SQLExecDirectW); [42S22] [Microsoft][ODBC Driver 17 for SQL Server][SQL Server]Invalid column name '1_7351907057963079'. (207); [42S22] [Microsoft][ODBC Driver 17 for SQL Server][SQL Server]Invalid column name '2_985074626865658'. (207); [42S22] [Microsoft][ODBC Driver 17 for SQL Server][SQL Server]Invalid column name '2_026901404032322'. (207); [42S22] [Microsoft][ODBC Driver 17 for SQL Server][SQL Server]Invalid column name 'Econ_Status'. (207); [42S22] [Microsoft][ODBC Driver 17 for SQL Server][SQL Server]Could not use view or function 'vw_Pizza_Model_Data' because of binding errors. (4413)")
[SQL: SELECT * FROM vw_Pizza_Model_Data]
(Background on this error at: https://sqlalche.me/e/20/f405)

👋 Model 1 Session Closed.


c:\Users\kelse\miniforge3\envs\ml-env\Lib\site-packages\pandas\io\sql.py:1649: SAWarning: Unrecognized server version info '17.0.1000.7'.  Some SQL Server features may not function properly.
  con = self.exit_stack.enter_context(con.connect())
